# Final Project: Automated solutions for algorithmic bias, Julia Portion
##### by: Cheng-Yu (Ben) Chiang from MATH 157 Winter 2022

## Limitations of Julia Language
Compared to Python, Julia's support for automated solutions for algorithmic bias are more rare and less extensive. Initially, I intended to use `PyCall` and import the related `aif360` functions. However, due to the lack of documentation on aif360's use in Julia I could not successfully import and use the package. After a bit of research, I stumbled across another community package [fairness.jl](https://github.com/ashryaagr/Fairness.jl) but couldn't successfully install the package due to lack of memory (Cocalc terminates my program whenever I try to install it). 

Therefore, as a workaround, we will explore different metrics of bias by calculating them on the adult dataset to gain a better understanding of the metrics that we will use in our python script later.

In [1]:
using CSV
using DataFrames

Firstly, lets load the dataset that was generated from predicting the label of the adult dataset using a standard logistic regression. This dataset contains the target, predicted values, and the gender of the samples. We need to first run a sample prediction because two of the metrics that we are going to discuss here examines the predicted values against the labels and we cannot do that with just the original dataset.
Note: the code that generates this data can be found in the python notebook's "Risk of Training with Biased Data" section

In [2]:
df = CSV.read("parsed_dataset/biased_results.csv", DataFrame)

,Column1,target,prediction,class_0_prob,class_1_prob,gender
,Int64,Int64,Int64,Float64,Float64,String
1,7762,0,0,0.995306,0.00469379,Male
2,23881,0,0,0.998258,0.00174155,Female
3,30507,0,0,0.996647,0.00335252,Male
4,28911,0,0,0.995548,0.0044521,Female
5,19484,0,0,0.972979,0.027021,Male
6,43031,0,0,0.587811,0.412189,Male
7,28188,0,0,0.758536,0.241464,Male
8,12761,0,0,0.896304,0.103696,Male
9,40834,0,0,0.713358,0.286642,Female


## Statistical Parity Difference
The bias metric measures the difference in the favorable outcome of two groups of population, one privileged and another unprivileged.

The idea is simple: group up two groups of people by a specific trait (gender, race, location, etc) and calculate their class conditional probability of getting a favorable outcome.

In our example of the Adult income dataset, the favorable outcome will be having >50K income and the unprivileged group will be female.

The probability equation can be formulated as: 
$$
\text{Statistical Parity Difference} = P( \text {favorable outcome} | \text{unprivileged}) - P( \text{favorable outcome} | \text{privileged})
$$

In [3]:
male_df = filter(row -> row.gender == "Male", df);
female_df = filter(row -> row.gender == "Female", df);

In [4]:
male_favorable_prob = sum(male_df.target) / nrow(male_df);
female_favorable_prob = sum(female_df.target) / nrow(female_df);
print("Statistical Parity Difference between female and male in terms of income: ", round(female_favorable_prob - male_favorable_prob, digits=5))

Statistical Parity Difference between female and male in terms of income: -0.19113

Interpretation:
- Ideal value = 0 
- Usually values between -0.1 and 0.1 is considered fair
- Measures which group is favored based on their probability of getting a good outcome (>50K)

## Equal Opportunity Difference
This metric computes the difference between the true positive rates of unprivileged and privileged groups. 
Note that true positive for group $A$ is defined as 
$$
\frac{P(TP|group=A)}{P(P|group=A)}
$$

For instance, if we want to find out the true positive rate of female in terms of our classification earlier in the Python notebook:
- filter the dataframe to include only female entries
- in those entries, count the number of rows where label = prediction = 1
- now count the number of actual positives, where the only condition is label = 1
- finally, we divide the two to obtain the *true positive rate*

$$
\frac{P(TP|group=F)}{P(P|group=F)} - \frac{P(TP|group=M)}{P(P|group=M)}
$$
where $F$ = female and $M$ = male

Interpretation of the metric
- The ideal outcome will be no difference (i.e. = 0), which means both groups have the same opporunity to get positive result. In our case that means both female and male have an equal chance at being classified as having above 50K income.
- values $<0$ means the model / system is biased towards benefitting the privileged group
- values $>0$ means the model / system is biased towards benefitting the unprivileged group
- Typcially, an acceptable (fair) range of the difference is $[-0.1, 0.1]$


In [5]:
actual_positive_male = size(filter(row -> row.target == 1, male_df))[1];
true_positive_male = size(filter(row -> row.target == 1 && row.prediction == 1, male_df))[1];

tp_rate_male = true_positive_male / actual_positive_male

0.6038808966209435

In [6]:
actual_positive_female = size(filter(row -> row.target == 1, female_df))[1];
true_positive_female = size(filter(row -> row.target == 1 && row.prediction == 1, female_df))[1];

tp_rate_female = true_positive_female / actual_positive_female

0.4900900900900901

In [7]:
# calculate the difference
print("Equal Opportunity Difference between female and male is: ", round(tp_rate_female - tp_rate_male, digits=5))

Equal Opportunity Difference between female and male is: -0.11379

How do we explain this result? 
- Female has less opportunity compared to male when we use our classifier to classify income
- The model classifies the positivity better in men compared to female.
- In other words, if the model is in charge of admitting students or selecting new employees, it is more likely that the model makes a mistake on female applicants. 
- Ideally, we want the model to perform the same under different genders (or other protected features). 

## Average Odds Difference 

Another metric that is closely related to the equal opportunity difference is the average odds difference. Average odds difference is defined as the average difference of the false positive rate and true positive rate between unprivileged and privileged groups.
This might be a little bit confusing, so let's try to write it in math

$$
{[P(FP| \text{unprivileged}) -P(FP|  \text{privileged})] + [ P(TP | \text{unprivileged})- P(TP |  \text{privileged})]} / 2
$$

In [8]:
actual_negative_male = size(filter(row -> row.target == 0, male_df))[1]; # negatives
fp_male = size(filter(row -> row.target == 0 && row.prediction == 1, male_df))[1]; # false positive

fp_rate_male = fp_male / actual_negative_male

0.09703028521023228

In [9]:
actual_negative_female = size(filter(row -> row.target == 0, female_df))[1];
fp_female = size(filter(row -> row.target == 0 && row.prediction == 1, female_df))[1];

fp_rate_female = fp_female / actual_negative_female

0.0162526120269329

In [10]:
println(fp_rate_female - fp_rate_male)
println(tp_rate_female - tp_rate_male)

-0.08077767318329938
-0.11379080653085338


In [11]:
print("Average Odds Difference between female and male is: ", round(((fp_rate_female - fp_rate_male) + (tp_rate_female - tp_rate_male))/2, digits=5))

Average Odds Difference between female and male is: -0.09728

Again, let's try to understand what is happening: 
- The false positive rate of male is significantly higher compared to female's.
- Implies that male is more likely to be misclassified in the >50K category while female's result is accurate
- Ideally, we want the false positive rate to be the same to show that there is no bias
- Intuitively: how much the unprivileged class is likely to be classified as positive (50>K)

### Now, let's head back to Python to see how we can rectify these biases.